In [1]:
# TODO consolidate all imports to agent0
# TODO wrap these calls in an interface for experiments
from ethpy.base import launch_local_chain
from ethpy.hyperdrive.api import HyperdriveInterface
from fixedpointmath import FixedPoint
from hypertypes.IHyperdriveTypes import Fees, PoolConfig
from web3.constants import ADDRESS_ZERO
from ethpy.hyperdrive import deploy_hyperdrive_from_factory
from ethpy import EthConfig
from agent0.hyperdrive.exec import async_fund_agents, create_and_fund_user_account
from agent0.base.make_key import make_private_key
from agent0.hyperdrive.agents import HyperdriveAgent
from eth_account.account import Account
from agent0 import AccountKeyConfig

In [2]:
# Hyperdrive initialization parameters

# ABI folder should contain JSON and Bytecode files for the following contracts:
# ERC20Mintable, MockERC4626, ForwarderFactory, ERC4626HyperdriveDeployer, ERC4626HyperdriveFactory
abi_dir = "../agent0/packages/hyperdrive/src/abis/"

# Deployer is the pre-funded account 0 on the Delv devnet
deployer_private_key: str = (
    "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"
)

# Factory initializaiton parameters
initial_variable_rate = FixedPoint("0.05")
curve_fee = FixedPoint("0.1")  # 10%
flat_fee = FixedPoint("0.0005")  # 0.05%
governance_fee = FixedPoint("0.15")  # 15%
max_curve_fee = FixedPoint("0.3")  # 30%
max_flat_fee = FixedPoint("0.0015")  # 0.15%
max_governance_fee = FixedPoint("0.30")  # 30%

# Pool initialization parameters
initial_fixed_rate = FixedPoint("0.05")  # 5%
initial_liquidity = FixedPoint(100_000_000)  # 100M ETH
initial_share_price = FixedPoint(1)
minimum_share_reserves = FixedPoint(10)
minimum_transaction_amount = FixedPoint("0.001")
precision_threshold = int(1e14)
time_stretch = (
    FixedPoint("0.04665") * (initial_fixed_rate * FixedPoint(100))
) / FixedPoint("5.24592")
position_duration = 604800  # 1 week
checkpoint_duration = 3600  # 1 hour
oracle_size = 10
update_gap = 3600  # 1 hour

# Derived values
fees = Fees(curve_fee.scaled_value, flat_fee.scaled_value, governance_fee.scaled_value)
max_fees = Fees(
    max_curve_fee.scaled_value,
    max_flat_fee.scaled_value,
    max_governance_fee.scaled_value,
)
initial_pool_config = PoolConfig(
    "",  # will be determined in the deploy function
    ADDRESS_ZERO,  # address(0), this address needs to be in a valid address format
    bytes(32),  # bytes32(0)
    initial_share_price.scaled_value,
    minimum_share_reserves.scaled_value,
    minimum_transaction_amount.scaled_value,
    precision_threshold,
    position_duration,
    checkpoint_duration,
    time_stretch.scaled_value,
    "",  # will be determined in the deploy function
    "",  # will be determined in the deploy function
    fees,
)

In [3]:
# Run local chain and deploy hyperdrive with parameters

# TODO this function is yielding the anvil address, where the return call
# does cleanup. Fix this to be a stateful object for launching a local chain in a subprocess.
anvil_address_gen = launch_local_chain(anvil_port=9999)
anvil_address = next(anvil_address_gen)

# Initialize the hyperdrive interface and deploy the chain
# TODO this function is deploying both the factory and the initial pool.
# Should expose both factory deployment and initializing a pool from the factory.
hyperdrive_chain = deploy_hyperdrive_from_factory(
    anvil_address,
    abi_dir,
    deployer_private_key,
    initial_liquidity,
    initial_variable_rate,
    initial_fixed_rate,
    initial_pool_config,
    max_fees,
)

hyperdrive_interface = HyperdriveInterface(
    EthConfig(artifacts_uri="not used", rpc_uri=anvil_address, abi_dir=abi_dir),
    hyperdrive_chain.hyperdrive_contract_addresses,
)




                             _   _
                            (_) | |
      __ _   _ __   __   __  _  | |
     / _` | | '_ \  \ \ / / | | | |
    | (_| | | | | |  \ V /  | | | |
     \__,_| |_| |_|   \_/   |_| |_|

    0.1.0 (08a629a 2023-06-03T00:15:12.657002000Z)
    https://github.com/foundry-rs/foundry

Available Accounts

(0) "0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266" (10000.000000000000000000 ETH)
(1) "0x70997970C51812dc3A010C7d01b50e0d17dc79C8" (10000.000000000000000000 ETH)
(2) "0x3C44CdDdB6a900fa2b585dd299e03d12FA4293BC" (10000.000000000000000000 ETH)
(3) "0x90F79bf6EB2c4f870365E785982E1f101E93b906" (10000.000000000000000000 ETH)
(4) "0x15d34AAf54267DB7D7c367839AAf71A00a2C6A65" (10000.000000000000000000 ETH)
(5) "0x9965507D1a55bcC2695C58ba16FB37d819B0A4dc" (10000.000000000000000000 ETH)
(6) "0x976EA74026E726554dB657fA54763abd0C3a0aa9" (10000.000000000000000000 ETH)
(7) "0x14dC79964da2C08b23698B3D3cc7Ca32193d9955" (10000.000000000000000000 ETH)
(8) "0x23618e81E3f5cdF7f54C3

/Users/slundquist/workspace/hyperdrive-experiments/.venv/lib/python3.10/site-packages/web3/contract/base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x075492621354d81a5d3a14047f557391867026e574ff3b6302e61e9d0bd84fc7') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/slundquist/workspace/hyperdrive-experiments/.venv/lib/python3.10/site-packages/web3/contract/base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x075492621354d81a5d3a14047f557391867026e574ff3b6302e61e9d0bd84fc7') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/slundquist/workspace/hyperdrive-experiments/.venv/lib/python3.10/site-packages/web3/contract/base_contract.py:185: UserWarning: The log with transaction ha

In [4]:
# Set up agents
num_agents = 1
eth_budgets = [
    1 * 10**18,
] * num_agents
base_budgets = [
    50_000 * 10**18,
] * num_agents

agents: list[HyperdriveAgent] = []
for _ in range(num_agents):
    user_private_key = make_private_key()
    agents.append(HyperdriveAgent(Account().from_key(user_private_key)))

account_config = AccountKeyConfig(
    deployer_private_key,
    [agent._private_key for agent in agents],  # pylint: disable=protected-access
    eth_budgets,
    base_budgets,
)

# Fund accounts
user_account = create_and_fund_user_account(hyperdrive_interface.eth_config, account_config, hyperdrive_chain.hyperdrive_contract_addresses)
await async_fund_agents(
    user_account,
    hyperdrive_interface.eth_config,
    account_config,
    hyperdrive_chain.hyperdrive_contract_addresses,
)

eth_chainId
eth_call
anvil_setBalance
eth_getTransactionCount
eth_chainId
eth_estimateGas
eth_getBlockByNumber
eth_chainId
eth_sendRawTransaction

    Transaction: 0xe7ef802f05a1b3ee680bfa1f87aa56af020a05b9a878df2cb705b1d352762b03
    Gas used: 51245

    Block Number: 11
    Block Hash: 0xf7ca4d9c96743e5c64318fa7956331446926ccd65835fa39822f2119dc4ef374
    Block Time: "Fri, 17 Nov 2023 00:27:33 +0000"

eth_getTransactionReceipt

23-11-16 16:27:25: WARNING: base.__init__:
Policy random number generator (rng) argument not set, using seed of `123`.
eth_getBalance
eth_chainId
eth_call

23-11-16 16:27:25: INFO: fund_agents.async_fund_agents:
Funding Eth
eth_getTransactionCount
eth_getTransactionCount
eth_chainId
eth_getBlockByNumber
eth_chainId
eth_estimateGas
eth_sendRawTransaction

    Transaction: 0x59196bb9734d7cdc4db72bbf99b7a1a976eda4dc33b7bbd02ebb7506b0e89830
    Gas used: 21000

    Block Number: 12
    Block Hash: 0x5afba2af23278af3804030dd78cc0e40a8793109011a735a229cd874baf801b7


In [5]:
# execute trades
lp_agent_index = 0
trade_result = hyperdrive_interface.async_add_liquidity(
    agent=agents[lp_agent_index],
    trade_amount=FixedPoint(1000),
    min_apr=FixedPoint("0.001"),
    max_apr=FixedPoint("1.0"),
)

# TODO either expose the non-async version of this function or wrap these calls in something
trade_result = await hyperdrive_interface.async_open_long(
    agent=agents[0], trade_amount=FixedPoint(100), slippage_tolerance=FixedPoint("0.1")
)

trade_result = await hyperdrive_interface.async_open_short(
    agent=agents[0], trade_amount=FixedPoint(10), slippage_tolerance=FixedPoint("0.1")
)

eth_getBlockByNumber
eth_chainId
eth_call
eth_chainId
eth_call
eth_chainId
eth_call
eth_chainId
eth_call
eth_chainId
eth_call
eth_chainId
eth_call
eth_chainId
eth_estimateGas
eth_chainId
eth_call


ContractCallException: Error in preview transaction